In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
import seaborn as sns
%matplotlib inline

In [ ]:
MONTHS_GEN = {
    1: "января",
    2: "февраля",
    3: "марта",
    4: "апреля",
    5: "мая",
    6: "июня",
    7: "июля",
    8: "августа",
    9: "сентября",
    10: "октября",
    11: "ноября",
    12: "декабря"
}

In [ ]:
data = pd.read_csv('weather_data.csv', sep=',').drop(['text'], axis='columns')

In [ ]:
data.head()

In [ ]:
month_list = list(MONTHS_GEN.values())
month_list.append('month')

In [ ]:
m_data = data.loc[:, month_list]

In [ ]:
m_data.head()

In [ ]:
m_data.info()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
m_x= m_data.drop('month', axis=1)
m_y = m_data['month'].astype('int')

In [ ]:
m_x.shape, m_y.shape

In [ ]:
m_x_train, m_x_valid, m_y_train, m_y_valid = train_test_split(m_x, m_y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [ ]:
m_x_train.shape, m_x_valid.shape

In [ ]:
first_tree = DecisionTreeClassifier(random_state=17)

In [ ]:
np.mean(cross_val_score(first_tree, m_x_train, m_y_train, cv=5))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, m_x_train, m_y_train, cv=5))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tree_params = {'max_depth': np.arange(1, 15), 'max_features':[.5, .7, 1]}

In [ ]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [ ]:
%%time
tree_grid.fit(m_x_train, m_y_train);

In [ ]:
tree_grid.best_score_, tree_grid.best_params_


In [ ]:
tree_valid_pred = tree_grid.predict(m_x_valid)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(m_y_valid, tree_valid_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(m_y_valid, tree_valid_pred)

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
second_tree = DecisionTreeClassifier(max_depth=12).fit(m_x_train, m_y_train)
second_tree.score(m_x_valid, m_y_valid)


In [ ]:
m_x_train.shape, m_x_valid.shape

In [ ]:
first_tree = DecisionTreeClassifier(random_state=17)

In [ ]:
np.mean(cross_val_score(first_tree, m_x_train, m_y_train, cv=5))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
first_knn = KNeighborsClassifier()
np.mean(cross_val_score(first_knn, m_x_train, m_y_train, cv=5))

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
tree_params = {'max_depth': np.arange(1, 15), 'max_features':[.5, .7, 1]}

In [ ]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [ ]:
%%time
tree_grid.fit(m_x_train, m_y_train);

In [ ]:
tree_grid.best_score_, tree_grid.best_params_


In [ ]:
tree_valid_pred = tree_grid.predict(m_x_valid)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(m_y_valid, tree_valid_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(m_y_valid, tree_valid_pred)

In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
second_tree = DecisionTreeClassifier(max_depth=12).fit(m_x_train, m_y_train)
second_tree.score(m_x_valid, m_y_valid)


In [ ]:
from utils import sentence_to_bagofwords
test_bag = sentence_to_bagofwords('Желтый уровень опасности. 4 сентября (пятница) местами по восточной половине республики ожидаются грозы.')

In [ ]:
t = second_tree.predict(test_bag)
print(t)